<a href="https://colab.research.google.com/github/michalrylko/decision-latency/blob/main/00_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Decision Latency Analytics - Project Overview
## Apache Airflow Pull Requests Case Study

This project analyzes **decision-making latency** using real-world data from
GitHub Pull Requests (PRs).

A Pull Request is treated as a proxy for a **decision process**:
- a proposal is submitted
- reviewers evaluate the change
- a final decision is made (merge or close)

This notebook covers **Stage 0: Data Extraction**.
Subsequent stages will focus on:
- exploratory data analysis (EDA)
- feature engineering
- predictive modeling
- decision design recommendations


In [23]:
# !ls -lh /content/decision-latency/notebooks


ls: cannot access '/content/decision-latency/notebooks': No such file or directory


# 1. Environment Setup & GitHub API Access

This section prepares the execution environment and configures access to the
GitHub REST API. It is intentionally grouped as a single logical step to make
the notebook easier to follow.

Specifically, this section covers:
- installation and import of required Python libraries
- configuration of runtime parameters controlling data extraction
- authentication setup for GitHub API access

Authentication is handled via a personal access token (`GITHUB_TOKEN`)
provided externally (e.g. environment variable or Colab Secrets), ensuring
that sensitive credentials are not stored in the notebook or repository.


In [ ]:
# Imports & Environment Setup

# Install dependencies (Colab-safe)
!pip -q install pandas requests tqdm

# Standard library
import os
import time
from pathlib import Path

# Third-party libraries
import requests
import pandas as pd
from tqdm.auto import tqdm

# Optional: Google Colab (only if running in Colab)
try:
    from google.colab import userdata  # type: ignore
except ImportError:
    userdata = None

# Authetication
def load_github_token():
    # 1) Environment variable (local / GitHub Actions)
    token = os.getenv("GITHUB_TOKEN")
    if token:
        return token

    # 2) Google Colab Secrets fallback
    if userdata is not None:
        return userdata.get("GITHUB_TOKEN")

    return None


GITHUB_TOKEN = load_github_token()
print("GitHub token loaded:", bool(GITHUB_TOKEN))

GitHub token loaded: True


If the token is correctly configured, authenticated requests will benefit from
a significantly higher rate limit (~5000 requests/hour).

# 2. Repository Configuration

We extract data from the public GitHub repository:

- Repository: `apache/airflow`
- Data source: GitHub Pull Requests API
- Scope: closed Pull Requests only

In [ ]:
# GitHub repository
REPO_OWNER = "apache"
REPO_NAME = "airflow"

# API behavior
PER_PAGE = 100          # GitHub API maximum
MAX_PAGES = 5           # Controls dataset size (~500 PRs)
REQUEST_DELAY = 0.2     # Seconds between API requests

# Checkpointing
CHECKPOINT_EVERY = 100  # Save progress every N PRs

headers = {"Accept": "application/vnd.github+json"}
if GITHUB_TOKEN:
    headers["Authorization"] = f"Bearer {GITHUB_TOKEN}"

session = requests.Session()
session.headers.update(headers)


# 3. GitHub API Helper

A helper function is used to:
- handle temporary API failures
- retry requests affected by rate limits
- keep the extraction pipeline robust


In [ ]:
def fetch_json(url, params=None, retries=3):
    for attempt in range(retries):
        response = session.get(url, params=params)

        if response.status_code == 200:
            return response.json()

        if response.status_code in (403, 429, 502, 503, 504):
            wait_time = 2 ** attempt
            time.sleep(wait_time)
            continue

        print(f"API error {response.status_code}: {response.text[:200]}")
        return None

    return None


## 4. Pull Request Collection

In this step, we collect Pull Request data in a **two-stage process** designed
to be robust and resumable for large-scale extraction.

**Stage 1 — Lightweight PR Index**
- Fetch a paginated list of closed Pull Requests
- Capture basic identifiers and timestamps
- Minimize API load in the initial pass

**Stage 2 — Detailed PR Metadata**
- Retrieve detailed metadata for each Pull Request
- Enrich records with review activity and change metrics
- Persist intermediate results using checkpointing to avoid data loss

The collected features include:
- number of comments and review comments
- commit count
- files changed
- lines added and deleted

Progress indicators and periodic checkpoints ensure that the extraction process
can safely resume in case of interruptions.

In [ ]:
print("DEBUG MAX_PAGES =", MAX_PAGES, "PER_PAGE =", PER_PAGE)

BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
DATA_DIR.mkdir(exist_ok=True)

FINAL_CSV = DATA_DIR / "apache_airflow_pull_requests_raw.csv"
CHECKPOINT_CSV = DATA_DIR / "apache_airflow_pull_requests_checkpoint.csv"


# Step 1: Lightweight Pull Request index
pull_requests_light = []

for page in tqdm(range(1, MAX_PAGES + 1), desc="Fetching PR pages"):
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/pulls"
    params = {
        "state": "closed",
        "per_page": PER_PAGE,
        "page": page
    }

    data = fetch_json(url, params=params)
    if not data:
        break

    for pr in data:
        pull_requests_light.append({
            "pr_number": pr.get("number"),
            "pr_id": pr.get("id"),
            "repository": f"{REPO_OWNER}/{REPO_NAME}",
            "created_at": pr.get("created_at"),
            "closed_at": pr.get("closed_at"),
            "merged_at": pr.get("merged_at"),
            "author": (pr.get("user") or {}).get("login"),
            "labels": [label.get("name") for label in pr.get("labels", [])]
        })

    time.sleep(REQUEST_DELAY)

print(f"Collected {len(pull_requests_light)} lightweight PR records")


# Scope alignment for checkpoint resume
light_pr_numbers = {
    pr["pr_number"]
    for pr in pull_requests_light
    if pr.get("pr_number") is not None
}
total_prs = len(light_pr_numbers)


# Step 2: Detailed Pull Request metadata
pull_requests_full = []
processed_pr_numbers = set()

if CHECKPOINT_CSV.exists():
    try:
        existing = pd.read_csv(CHECKPOINT_CSV)
        existing = existing[existing["pr_number"].isin(light_pr_numbers)]

        pull_requests_full = existing.to_dict(orient="records")
        processed_pr_numbers = set(
            existing["pr_number"].dropna().astype(int).tolist()
        )

        print(
            f"Resuming from checkpoint: "
            f"{len(processed_pr_numbers)} PRs already processed"
        )
    except Exception as e:
        print(f"Checkpoint could not be read ({e}). Starting fresh.")

saved_at = len(processed_pr_numbers)

for pr in tqdm(pull_requests_light, desc="Fetching PR details"):
    pr_number = pr.get("pr_number")
    if pr_number is None or pr_number in processed_pr_numbers:
        continue

    detail_url = (
        f"https://api.github.com/repos/"
        f"{REPO_OWNER}/{REPO_NAME}/pulls/{pr_number}"
    )

    details = fetch_json(detail_url)
    if details is None:
        continue

    pull_requests_full.append({
        **pr,
        "merged": details.get("merged_at") is not None,
        "comments_count": details.get("comments"),
        "review_comments_count": details.get("review_comments"),
        "commit_count": details.get("commits"),
        "changed_files_count": details.get("changed_files"),
        "additions": details.get("additions"),
        "deletions": details.get("deletions")
    })

    processed_pr_numbers.add(pr_number)

    if (
        len(processed_pr_numbers) % 50 == 0
        or len(processed_pr_numbers) == total_prs
    ):
        print(f"Processed {len(processed_pr_numbers)}/{total_prs} PRs")

    if len(processed_pr_numbers) - saved_at >= CHECKPOINT_EVERY:
        df_ckpt = pd.DataFrame(pull_requests_full)
        tmp_path = CHECKPOINT_CSV.with_suffix(".tmp")
        df_ckpt.to_csv(tmp_path, index=False)
        tmp_path.replace(CHECKPOINT_CSV)
        saved_at = len(processed_pr_numbers)

        print(f"Checkpoint saved: {saved_at} PRs")

    time.sleep(REQUEST_DELAY)


print(f"Collected {len(pull_requests_full)} detailed PR records")

df = pd.DataFrame(pull_requests_full)
df.to_csv(FINAL_CSV, index=False)

print(f"Final dataset saved: {len(df)} rows -> {FINAL_CSV}")

DEBUG MAX_PAGES = 5 PER_PAGE = 100


Fetching PR pages:   0%|          | 0/5 [00:00<?, ?it/s]

Collected 500 lightweight PR records
Resuming from checkpoint: 499 PRs already processed


Fetching PR details:   0%|          | 0/500 [00:00<?, ?it/s]

Processed 500/500 PRs
Collected 500 detailed PR records
Final dataset saved: 500 rows -> /data/apache_airflow_pull_requests_raw.csv


## 5. Dataset Construction & Decision Latency Metric

Decision latency is defined as the number of days between **Pull Request creation** and **Pull Request closure** (merge or close).


In [ ]:
df = pd.DataFrame(pull_requests_full)

df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")
df["closed_at"] = pd.to_datetime(df["closed_at"], errors="coerce")

df["decision_latency_days"] = (df["closed_at"] - df["created_at"]).dt.days

# Basic sanity cleanup
df = df[df["decision_latency_days"].notna() & (df["decision_latency_days"] >= 0)]

df.head()

,pr_number,pr_id,repository,created_at,closed_at,merged_at,author,labels,merged,comments_count,review_comments_count,commit_count,changed_files_count,additions,deletions,decision_latency_days
0,59410,3099737196,apache/airflow,2025-12-14 12:23:16+00:00,2025-12-14 12:24:54+00:00,2025-12-14T12:24:54Z,github-actions[bot],['kind:documentation'],True,0,0,1,1,98,0,0
1,59404,3099475610,apache/airflow,2025-12-14 07:57:14+00:00,2025-12-14 09:42:17+00:00,NaN,github-actions[bot],"['area:dev-tools', 'backport-to-v3-1-test']",False,0,0,1,1,1,1,0
2,59403,3099446203,apache/airflow,2025-12-14 07:13:16+00:00,2025-12-14 07:56:07+00:00,2025-12-14T07:56:07Z,choo121600,"['area:dev-tools', 'backport-to-v3-1-test']",True,2,0,1,1,1,1,0
3,59401,3099329724,apache/airflow,2025-12-14 04:34:52+00:00,2025-12-14 07:48:57+00:00,2025-12-14T07:48:57Z,choo121600,['area:UI'],True,0,0,2,2,9,9,0
4,59397,3099260718,apache/airflow,2025-12-14 02:59:16+00:00,2025-12-14 07:37:06+00:00,NaN,github-actions[bot],"['area:dev-tools', 'backport-to-v3-1-test']",False,1,0,1,2,50,0,0


# 6. Export Validation

At this stage, the dataset has already been persisted during extraction.
Here we only validate the saved artifact and confirm basic dataset statistics.

In [ ]:
if not FINAL_CSV.exists():
    raise FileNotFoundError(f"Expected dataset file not found: {FINAL_CSV}")

df_saved = pd.read_csv(FINAL_CSV)

# Ensure timestamps are parsed
df_saved["created_at"] = pd.to_datetime(df_saved.get("created_at"), errors="coerce")
df_saved["closed_at"] = pd.to_datetime(df_saved.get("closed_at"), errors="coerce")

# Ensure decision_latency_days exists (recompute if missing)
if "decision_latency_days" not in df_saved.columns:
    df_saved["decision_latency_days"] = (df_saved["closed_at"] - df_saved["created_at"]).dt.days

# Basic sanity cleanup
df_saved = df_saved[df_saved["decision_latency_days"].notna() & (df_saved["decision_latency_days"] >= 0)]

print("Saved dataset shape:", df_saved.shape)
display(df_saved.head())
print()
df_saved["decision_latency_days"].describe()


Saved dataset shape: (500, 16)


,pr_number,pr_id,repository,created_at,closed_at,merged_at,author,labels,merged,comments_count,review_comments_count,commit_count,changed_files_count,additions,deletions,decision_latency_days
0,59410,3099737196,apache/airflow,2025-12-14 12:23:16+00:00,2025-12-14 12:24:54+00:00,2025-12-14T12:24:54Z,github-actions[bot],['kind:documentation'],True,0,0,1,1,98,0,0
1,59404,3099475610,apache/airflow,2025-12-14 07:57:14+00:00,2025-12-14 09:42:17+00:00,NaN,github-actions[bot],"['area:dev-tools', 'backport-to-v3-1-test']",False,0,0,1,1,1,1,0
2,59403,3099446203,apache/airflow,2025-12-14 07:13:16+00:00,2025-12-14 07:56:07+00:00,2025-12-14T07:56:07Z,choo121600,"['area:dev-tools', 'backport-to-v3-1-test']",True,2,0,1,1,1,1,0
3,59401,3099329724,apache/airflow,2025-12-14 04:34:52+00:00,2025-12-14 07:48:57+00:00,2025-12-14T07:48:57Z,choo121600,['area:UI'],True,0,0,2,2,9,9,0
4,59397,3099260718,apache/airflow,2025-12-14 02:59:16+00:00,2025-12-14 07:37:06+00:00,NaN,github-actions[bot],"['area:dev-tools', 'backport-to-v3-1-test']",False,1,0,1,2,50,0,0


,decision_latency_days
count,500.000000
mean,0.898000
std,1.923277
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,11.000000


# New Section

# 7. Summary & Next Steps

This notebook completed **Stage 0: Data Extraction**.

Next stages:
- Exploratory Data Analysis (EDA)
- Feature engineering
- Predictive modeling
- Decision design recommendations
